In [ ]:
from cag.framework.annotator.pipeline.pipeline_base import Pipeline
from cag.utils.config import Config
from tqdm import tqdm
from cag.graph_elements.base_graph import BaseGraph
from pyArango.graph import  EdgeDefinition
import cag


In [ ]:
# make sure arangodb is up and running - Enter your credentials below
my_config = Config(
            url= "http://127.0.0.1:8529",
            user= "root",
            password= "oiE9O6s14rLwJ2SB",
            database= "_system",
            graph= "MyFirstCagGraph"
        )


In [6]:
class MyFirstPipeline(Pipeline):
    def __init__(self, database_config: Config, batch_size:int = 50):
        super().__init__(database_config)
        self.period = None
        self.batch_size = batch_size
    
    def process_input(self) -> list:
        processed = []
        for txt_node in tqdm(self.input):
            processed.append((txt_node.text, {"_key": txt_node._key}))

        return processed
    
    
    def init_and_run(self) -> list:

        # 3. Loop over your data, annotate and save
        coll = self.database_config.db["TextNode"]
        docs = coll.fetchAll(limit=self.batch_size)
        fetched = len(docs)

        # set the pipeline by adding pre defined pipelines from CAG or using cunstomized ones
        #self.set_spacy_language_model("en_core_web_md")
        self.add_annotation_pipe(name="NamedEntityPipeOrchestrator", save_output=True, is_spacy=True) # spacy 
        self.add_annotation_pipe(name="EmpathPipeOrchestrator", save_output=True, is_spacy=True)

        while docs != None and len(docs) >0:
            ## annotate

            # Set the INPUT - this will automatically call preprocess_input (make sure to implement it)
            self.set_input(docs)
            
            self.annotate()
            self.save()
            
            self.reset_input_output()
            print(f"Processed {fetched} docs")
            docs = coll.fetchAll(limit=self.batch_size, skip=fetched)
            fetched = fetched + len(docs)
            

In [7]:
my_first_pipeline = MyFirstPipeline(my_config, batch_size =10)

In [ ]:
my_first_pipeline.init_and_run()